In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error

from sklearn.ensemble import ExtraTreesRegressor

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("../data/final/datos_procesados.csv")

In [4]:
df = df[df['donation_type'] == 1]

In [5]:
df = df.groupby('donation_date').sum()

In [6]:
df['fecha'] = pd.to_datetime(df.index)

In [7]:
fecha_inicio = df.index.min()
fecha_fin = df.index.max()
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

In [8]:
# Crear un DataFrame con todas las fechas posibles
serie_temporal = pd.DataFrame()
serie_temporal['fecha'] = rango_fechas

In [9]:
# Unir los datos de donaciones al DataFrame de fechas
serie_temporal = pd.merge(serie_temporal, df, on='fecha', how='left')

In [10]:
# Rellenar los valores faltantes con 0
serie_temporal['amount'].fillna(0, inplace=True)

In [11]:
serie_temporal['fecha'] = pd.to_datetime(serie_temporal['fecha'])
serie_temporal['año'] = serie_temporal['fecha'].dt.year
serie_temporal['mes'] = serie_temporal['fecha'].dt.month
serie_temporal['dia'] = serie_temporal['fecha'].dt.day

In [12]:
serie_temporal = serie_temporal.drop(['name', 'fecha', 'email', 'date', 'is_partner', 'company', 'role_company', 'donation_frecuency', 'suscription_status', 'donation_type', 'method_pay'], axis=1)

In [13]:
# Dividir los datos en características (X) y la variable objetivo (y)
X = serie_temporal[['año', 'mes', 'dia']]
y = serie_temporal['amount']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Crear el modelo ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=100, random_state=42)

In [15]:
# Entrenar el modelo
model.fit(X_train, y_train)

ExtraTreesRegressor(random_state=42)

In [16]:
X_train

,año,mes,dia
1699,2011,7,25
57,2007,1,25
3641,2016,11,17
3039,2015,3,26
6302,2024,3,1
...,...,...,...
3772,2017,3,28
5191,2021,2,14
5226,2021,3,21
5390,2021,9,1


In [17]:
# Hacer predicciones en el conjunto de prueba
predictions = model.predict(X_test)

In [18]:
# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, predictions)
print("RMSE:", np.sqrt(mse))
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación R^2:", r2)

RMSE: 1683.9346101999533
Coeficiente de determinación R^2: 0.7253699574264386


In [19]:
# import pickle

# # Guarda el modelo en un archivo pickle
# with open('../model/extra_tree_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

# df_vacio = X_train.drop(X_train.index)
# # Guarda un dataframe vacío
# with open('../model/dataframe.pkl', 'wb') as f:
#     pickle.dump(df_vacio, f)

In [20]:
print("MAE", mean_absolute_error(y_test, predictions))
print("MAPE", mean_absolute_percentage_error(y_test, predictions))

MAE 867.4479699457142
MAPE 1.5460500596592443e+18
